## Decay spectra.

### dndE_decay spectrum

In [ ]:
### Energy Range.
X_array = jnp.linspace(-40, 80, 100)*Params.meV  # K_e - K0_end (meV)
E_e_array = X_array + Params.m_e + Params.K0_end
E_j_array = Ptolemy.E_j_from_E_e(E_e_array, Params())

### Initial masses & lifetimes.
m_lightest = 0.*Params.meV
ordering = 'NO'
nu_masses = Physics.neutrino_masses(m_lightest, ordering, Params())
m_j, _, m_i = nu_masses
tau_i = 2*Params.t0
n0_i = 56/Params.cm**3
m_x = 0.*Params.meV

dnj_dEj = jnp.array([
    Ptolemy.dndE_decay(E, m_j, m_i, m_x, tau_i, n0_i, Params()) for E in E_j_array])

y_axis = dnj_dEj*(Params.eV*Params.cm**2)
plt.plot(E_j_array/Params.meV, y_axis)
plt.show(); plt.close()

### Recreation of Fig. 3 of Akita et al. (2022)

In [ ]:
# note: I learned from constructing the PTOLEMY spectra, that you need to
# do proper normalizations for your convolution kernels, and to calculate the 
# spectra and convolutions for a larger "x-axis" range, and then truncate the 
# plot to the x-axis range you want to show, such that you avoid or make the
# (unphysical) edge effects of the convolution procedure invisible in the plot.

### Energy Range.
X_array = jnp.linspace(-40, 80, 100)*Params.meV  # K_e - K0_end (meV)
E_e_array = X_array + Params.m_e + Params.K0_end

### Initial masses & lifetimes.
m_lightest = 0.*Params.meV
ordering = 'NO'
nu_masses = Physics.neutrino_masses(m_lightest, ordering, Params())
m_j, _, m_i = nu_masses
tau_i = 2*Params.t0
n0_i = 56/Params.cm**3

### Original beta-decay spectrum.
beta_spec = Ptolemy.Beta_decay_spectrum(E_e_array, nu_masses, ordering, Params())

### Convolved beta-decay spectrum.
Delta = 10*Params.meV
sigma = Delta / jnp.sqrt(8*jnp.log(2))

# Normalization factor for convolution.
conv_norm = jnp.sqrt(2*Params.Pi)*sigma

convolved_beta_spec = jnp.zeros(len(E_e_array))
for i, E_e in enumerate(E_e_array):
    
    # Integration range.
    E_e_prime = E_e_array
    
    # Functions to convolve.
    f1 = Ptolemy.Beta_decay_spectrum(E_e_array, nu_masses, ordering, Params())
    g1 = jnp.exp(-(E_e - E_e_prime)**2/(2*sigma**2))

    # Convolved value at E_e.
    jax_set = jsp.integrate.trapezoid(f1*g1, x=E_e_prime) / conv_norm
    convolved_beta_spec = convolved_beta_spec.at[i].set(jax_set)


########################
### Electron spectra ###
########################

fig, ax = plt.subplots(figsize=(8,6))


#------------------------#
#-- Beta-decay spectra --#
#------------------------#

# ax.semilogy(X_array/meV, beta_spec*(yr*eV), ls=':', label='Bkg')
ax.semilogy(
    X_array/Params.meV, convolved_beta_spec*(Params.eV*Params.yr), 
    alpha=0.7, c='k', ls='-.', label=r'$\beta$-decay bkg'
)


#-------------------#
#-- Decay spectra --#
#-------------------#

# ax.semilogy(X_array/meV, decay_spec*(yr*eV), alpha=0.7, label=r'$\nu_1$ original')


# For upper panels.
mediator_masses = [0.*Params.meV, 20.*Params.meV, 40.*Params.meV]
colors = ['dodgerblue', 'orangered', 'gold']

# For lower panels.
# mediator_masses = [0.*meV]
# colors = ['dodgerblue']

for m_x, col in zip(mediator_masses, colors):

    ### Spectrum of lighter neutrinos produced by the heavier neutrino decays.
    decay_spec = Ptolemy.dGammaTilde_CNB_dE_e(
        E_e_array, m_j, m_i, m_x, tau_i, n0_i, ordering, Params())


    convolved_decay_spec = jnp.zeros(len(E_e_array))
    for i, E_e in enumerate(E_e_array):
        
        # Integration range.
        E_e_prime = E_e_array
        
        # Functions to convolve.
        f1 = Ptolemy.dGammaTilde_CNB_dE_e(
            E_e_array, m_j, m_i, m_x, tau_i, n0_i, ordering, Params())
        g1 = jnp.exp(-(E_e - E_e_prime)**2/(2*sigma**2))
 
        # Convolved value at E_e.
        jax_set = jsp.integrate.trapezoid(f1*g1, x=E_e_prime) / conv_norm
        convolved_decay_spec = convolved_decay_spec.at[i].set(jax_set)

    ax.semilogy(
        X_array/Params.meV, convolved_decay_spec*(Params.yr*Params.eV), 
        c=col, label=fr'$\nu_1$ ($m_{{\phi}}={m_x/Params.meV}$ meV)'
    )


ax.set_title(r'Fig. 3 (top left panel)')
ax.set_xlabel(r'$K_e - K_{end}^0$ (meV)')
ax.set_ylabel(r'$d\Gamma / dE_e$ $(yr^{-1} eV^{-1})$')

plt.ylim(1e0, 1e12)
plt.legend()
plt.show(); plt.close()